In [1]:
import numpy as np
import pandas as pd
from groq import Groq
import time

In [2]:
df = pd.read_excel('../../data/raw/OMIEC_NEUROMORPHIC_20_08_24_v2.xlsx')

In [3]:
df['Abstract']

0      Organic electrochemical transistors are a prom...
1      Organic mixed ionic-electronic conductors (OMI...
2      Organic mixed ionic-electronic conductors (OMI...
3      The field of organic mixed ionic-electronic co...
4      Operational stability underpins the successful...
                             ...                        
118    In recent years, organic electrochemical trans...
119    Organic electrochemical transistors (OECTs) ma...
120    Owing to their potential advantages such as lo...
121    Due to similar transmission characteristics of...
122    Synaptic plasticity functions play a crucial r...
Name: Abstract, Length: 123, dtype: object

In [4]:
GROQ_API_KEY='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'

In [5]:
# groq
client = Groq(
    api_key=GROQ_API_KEY,
)

# respostas
responses_list = []

# todos os abstracts
total = len(df['Abstract'])
total_time = 0


for index, row in df.iterrows():
    start_time = time.time()

    # tempo
    print('\n', index, 'of', total, 'remaining estimated time', (total-index)*(total_time/(index+1)))

    # chat request
    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
             {
                "role": "system",
                "content": "RolePlay as a bot seeking for Organic Mixed Ionic Electronic Conductor=s (OMIEC) polymers that are used as active material in Organic Electrochemical Transistors (OECT) and in neuromorphic applications."
            },
            {
                "role": "user",
                "content": f"Answer 'Yes' or 'No' only. Does the following text mention any particular Organic Mixed Ionic Electronic Conductors (OMIEC) polymers that are used as active material in Organic Electrochemical Transistors (OECT) and in neuromorphic applications? Do not consider dopants, solvents, or catalysts. {row['Abstract']}"
        
            },
            {
                "role": "user",
                "content": "There is a possibility that the data you extracted is incorrect. Verify if any other OMIEC-type polymers are mentioned and ensure all have been identified."
            },
            {
                "role": "user",
                "content": f"If 'No', output only 'None' and do not answer any other request. If 'Yes', output only all OMIEC-type, OECT and neuromorphic applicable polymer names mentioned in the text, separated by semicolons."  
            },
            {
                "role": "user",
                "content": "In the text, search for the application keywords of each polymer. Do not consider 'OMIEC' as an application. For each polymer, fill out the JSON template as follows: { 'Polymer Mentioned Name': 'exact_name_found_in_text', 'Polymer Chemical Abbreviation':'polymer_abbreviation_for_data_analysis', 'Polymer Application': 'application_keywords', 'Doped?:': 'p-doped_or_n-doped_or_not_doped'}. Fill the 'Polymer Application' field with only the relevant application keywords, separated by commas. Separate each JSON object with a semicolon. Ensure the output follows the specified JSON template."
            }
        ],
        temperature=0.2,
        max_tokens=1024,
        top_p=0.9,
        stream=True,
        stop=None,
    )

    # coletar respostas
    response = ""

    # Stream the completion result and print the content
    for chunk in completion:
        response += chunk.choices[0].delta.content or ""
        print(chunk.choices[0].delta.content or "", end="")

    print()
    
    # listas de respostas
    responses_list.append({ 'index': index, 'response': response })

    # tempo total
    total_time += time.time() - start_time
    


 0 of 123 remaining estimated time 0.0
{ 'Polymer Mentioned Name': 'polythiophene', 'Polymer Chemical Abbreviation':'PTh', 'Polymer Application': 'neuromorphic computing, healthcare, bioelectronic devices', 'Doped?:': 'p-doped' }; { 'Polymer Mentioned Name': 'naphthalene diimide-based polymer', 'Polymer Chemical Abbreviation':'NDI', 'Polymer Application': 'neuromorphic computing, healthcare, bioelectronic devices', 'Doped?:': 'n-doped' }

 1 of 123 remaining estimated time 53.96460938453674
{ 'Polymer Mentioned Name': 'melanin', 'Polymer Chemical Abbreviation':'Mel', 'Polymer Application': 'bioelectronics, neuromorphic computing, soft robotics, energy storage, organic electrochemical transistors, supercapacitors', 'Doped?:': 'not doped'}; { 'Polymer Mentioned Name': 'poly(3,4-ethylenedioxythiophene) polystyrene sulfonate', 'Polymer Chemical Abbreviation':'PEDOT:PSS', 'Polymer Application': 'organic electrochemical transistors, supercapacitors', 'Doped?:': 'not doped'}

 2 of 123 remai

In [1]:
# DataFrame
df_with_responses = pd.DataFrame(responses_list)

# salvar para .csv
df_with_responses.to_excel('../../dataframes/llama3-70B-OMIEC_NEUROMORPHIC030924_0.2_temp_dict2_RESPONSES.xlsx', index=False)

NameError: name 'pd' is not defined

In [16]:
df_with_responses

,index,response
0,0,"{ 'Polymer Mentioned Name': 'polythiophene', '..."
1,1,"{ 'Polymer Mentioned Name': 'melanin', 'Polyme..."
2,2,{ 'Polymer Mentioned Name': 'poly(3-[2-[2-(2-m...
3,3,{}
4,4,None
...,...,...
118,118,{}
119,119,{}
120,120,{ 'Polymer Mentioned Name': 'poly(3-hexylthiop...
121,121,{ 'Polymer Mentioned Name': 'poly(3-hexylthiop...


In [8]:
df_with_responses = df_with_responses[df_with_responses['response'] != 'None']

In [9]:
df_with_responses

,index,response
0,0,"{ 'Polymer Mentioned Name': 'polythiophene', '..."
1,1,"{ 'Polymer Mentioned Name': 'melanin', 'Polyme..."
2,2,{ 'Polymer Mentioned Name': 'poly(3-[2-[2-(2-m...
3,3,{}
5,5,{}
...,...,...
118,118,{}
119,119,{}
120,120,{ 'Polymer Mentioned Name': 'poly(3-hexylthiop...
121,121,{ 'Polymer Mentioned Name': 'poly(3-hexylthiop...


In [10]:
for token in df_with_responses['response']:
    print(token.split(';'))

["{ 'Polymer Mentioned Name': 'polythiophene', 'Polymer Chemical Abbreviation':'PTh', 'Polymer Application': 'neuromorphic computing, healthcare, bioelectronic devices', 'Doped?:': 'p-doped' }", " { 'Polymer Mentioned Name': 'naphthalene diimide-based polymer', 'Polymer Chemical Abbreviation':'NDI', 'Polymer Application': 'neuromorphic computing, healthcare, bioelectronic devices', 'Doped?:': 'n-doped' }"]
["{ 'Polymer Mentioned Name': 'melanin', 'Polymer Chemical Abbreviation':'Mel', 'Polymer Application': 'bioelectronics, neuromorphic computing, soft robotics, energy storage, organic electrochemical transistors, supercapacitors', 'Doped?:': 'not doped'}", " { 'Polymer Mentioned Name': 'poly(3,4-ethylenedioxythiophene) polystyrene sulfonate', 'Polymer Chemical Abbreviation':'PEDOT:PSS', 'Polymer Application': 'organic electrochemical transistors, supercapacitors', 'Doped?:': 'not doped'}"]
["{ 'Polymer Mentioned Name': 'poly(3-[2-[2-(2-methoxyethoxy)ethoxy]ethyl]thiophene -2,5-diyl)',

#### Playground

In [11]:
df_with_responses

,index,response
0,0,"{ 'Polymer Mentioned Name': 'polythiophene', '..."
1,1,"{ 'Polymer Mentioned Name': 'melanin', 'Polyme..."
2,2,{ 'Polymer Mentioned Name': 'poly(3-[2-[2-(2-m...
3,3,{}
5,5,{}
...,...,...
118,118,{}
119,119,{}
120,120,{ 'Polymer Mentioned Name': 'poly(3-hexylthiop...
121,121,{ 'Polymer Mentioned Name': 'poly(3-hexylthiop...


In [12]:
result = pd.concat([df_with_responses, df], axis=1)
result.drop('index', axis=1, inplace=True)
result = result[result['response'] != 'None']
result = result.reset_index(drop=True)

In [13]:
result.to_excel('1_OMIEC_RESPONSES_output.xlsx', index=False)

In [14]:
polímeros = []
for response in result['response']:
    polímeros = polímeros + response.split(';')

AttributeError: 'float' object has no attribute 'split'

In [ ]:
polímeros_processado = [s.strip() for s in polímeros]

In [ ]:
serie_polímeros = pd.Series(polímeros_processado)
polímeros_filtrados = serie_polímeros[~serie_polímeros.str.contains(r'OMIEC.*', regex=True)]
polímeros_filtrados = polímeros_filtrados[polímeros_filtrados != '']
polímeros_filtrados = polímeros_filtrados[~polímeros_filtrados.str.contains(r'(?i)\bnone\b', regex=True)]
polímeros_filtrados = polímeros_filtrados[~polímeros_filtrados.str.contains('conduct')]
polímeros_filtrados = polímeros_filtrados[~polímeros_filtrados.str.contains('conjug')]
polímeros_filtrados = polímeros_filtrados[~polímeros_filtrados.str.contains('cellul')]
polímeros_filtrados = polímeros_filtrados[~polímeros_filtrados.str.contains(r'(?i)\bpolymer\b', regex=True)]
polímeros_filtrados = polímeros_filtrados[~polímeros_filtrados.str.contains(r'(?i)\bmixed\b', regex=True)]

In [ ]:
replacements = {
    r'.*\((PEDOT:PSS)\)': r'\1',
    r'.*\((PEDOT)\)': r'\1',
    r'.*\((PEDOT-P)\)': r'\1',
    r'.*\((PEDOT-T)\)': r'\1',
    r'poly\(3,4-ethylenedioxythiophene\)': 'PEDOT',
    r'.*\((P3HT)\)': r'\1',
    r'.*\((PVDF)\)': r'\1',
    r'PEDOT:polystyrenesulfonate': 'PEDOT:PSS',
    r'PEDOT:polystyrene sulfonate': 'PEDOT:PSS',
    r'PEDOT\s*:\s*PSS': 'PEDOT:PSS',
    r'PEDOT\/PSS': 'PEDOT:PSS',
    r'PEDOT\/?poly\(4-styrenesulfonate\)': 'PEDOT:PSS',
    r'PEDOT\s*[-]\s*PSS': 'PEDOT:PSS'


}

In [ ]:
for pattern, replacement in replacements.items():
    polímeros_filtrados = polímeros_filtrados.str.replace(pattern, replacement, regex=True, case=False)

In [ ]:
len(serie_polímeros)

683

In [ ]:
len(polímeros_filtrados.unique())

432

In [ ]:
polímeros_filtrados.unique()

array(['PEDOT', 'PEDOT:PolyDADMA', 'PEDOT:PolyDADMA FSI',
       'PEDOT:PolyDADMA TFSI', 'PEDOT:PSS', 'PSS', 'PEO-PEDOT',
       'PANI/PSS', 'poly( vinylferrocene) (PVF)', 'P(3-MT-TTF)', 'PTFE',
       'P(VDF-TrFE)', 'PBTTT', 'PVDF', 'P3HT', 'F4TCNQ', 'BPEEPTF', 'TTF',
       'Bipolar polymers.', 'PEDOT-T', 'PEDOT-P',
       "Poly(3,3'-dialkylimidazolium triflate)",
       'Poly(tetraphenylphosphonium tetracyanometate)',
       'Poly(3-methylthiophene)', 'Poly(o-phenylenediamine)',
       'Poly(3-phenyl-phenylene sulfide)', 'EDOTS', 'EDOTCOOH', 'PProDOT',
       'Polymerized ionic liquid', 'PMDT-TTF', 'TEDFE', 'PT-CSA-O-PTH',
       'PT-TSPT', 'HPTS-PDBT.', 'EG/GOPS-PEDOT:PSS', 'pg2T-TT',
       'f-BTI2g-TVTF', 'f-BTI2g-TVTCl', 'Polyparaphenylene',
       'Ni-3(HITP)(2)', 'PEDOT-Cl', '[C(2)mpyr][FSI]', 'poly-FADS',
       'gNR-Bu', 'PPSS', 'PPTFSI', 'f-BTI2g-SVSCN', 'f-BSeI2g-SVSCN',
       'PyQt', 'PBTTT-C8', 'OMEC-5', '(C16H33CH2CH2CO2)2Sn', 'DTPT',
       'PTAA-DMB', 'OMEM', 'OMEC-3

In [ ]:
df_polímeros = pd.DataFrame(polímeros_filtrados, columns=['Polímeros'])

In [ ]:
contagem = df_polímeros['Polímeros'].value_counts()

In [ ]:
contagem.head(30)

Polímeros
PEDOT:PSS               65
PEDOT                   34
P3HT                    18
TTF-CA                   6
PEO                      6
polythiophene            4
PANI                     4
MEH-PPV                  4
BBL                      3
PSS                      3
PTAA                     3
Polyaniline              3
PPV                      2
Polypyrrole              2
P3MEEET                  2
LiTFSI                   2
CH3NH3PbI3               2
poly(ethylene oxide)     2
Homo-gDPP                2
NDI-T2                   2
PSSNa                    2
Spiro-OMeTAD             2
PPy                      2
g4T2-T2                  2
(BEDT-TTF)               2
PTEO                     2
Polythiophene            2
PTh                      2
p(g2T-T)                 2
P3T                      2
Name: count, dtype: int64

In [ ]:
np.unique(df_polímeros[df_polímeros['Polímeros'].str.contains(r'PEDOT', regex=True)].values.tolist())


array(['EG/GOPS-PEDOT:PSS', 'PEDOT', 'PEDOT-Cl', 'PEDOT-MoS2', 'PEDOT-P',
       'PEDOT-PBA', 'PEDOT-T', 'PEDOT-b-PEG',
       'PEDOT-cobalt(II) hexylridge', 'PEDOT-silver perchlorate',
       'PEDOT:PSS', 'PEDOT:PSS.', 'PEDOT:PSS/SPEEK', 'PEDOT:PolyDADMA',
       'PEDOT:PolyDADMA FSI', 'PEDOT:PolyDADMA TFSI', 'PEDOT:Tos',
       'PEO-PEDOT', 'МIP-PEDOT-PBA'], dtype='<U27')

In [ ]:
df_polímeros.to_excel('1_OMIEC_RESPONSES_output_preprocessed.xlsx', index=False)

In [ ]:
df